# Simple GenAI App Using Langchain 

### load data --> Docs ---> divided our text into chaunks---> vecotre-->vectore Embedding----> vectore store in DB

### General Rule to Identify Module

- 1. **langchain_core (CORE LAYER)
Iska kaam: Sirf basic building blocks provide karna, jo LangChain ke andar hi kaam karte hain.**

- 2. **langchain_community (INTEGRATIONS LAYER)
Iska kaam: Saare external integrations handle karna.**

- 3. **langchain (HIGH-LEVEL LAYER)
Iska kaam: Sirf user-friendly wrappers provide karna jo core + community ko combine karte hain.**
    - **Agar integration hai (jaise API, database, file format) → langchain_community**
    - **Agar prompt, chain, schema, message → langchain_core**
    - **Agar high-level chains ya wrappers → langchain**



In [42]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [43]:
### Data Ingestion = from the website scrape tha data
from langchain_community.document_loaders import WebBaseLoader

In [44]:
loader = WebBaseLoader("https://docs.smith.langchain.com/observability/tutorials/observability")
loader

In [45]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/observability/tutorials/observability', 'title': 'Add observability to your LLM application | 🦜️🛠️ LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content='\n\n\n\n\nAdd observability to your LLM application | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityQuick StartTutorialsAdd observability to your LLM applicationHow-to GuidesAnnotate code for tracingFilter traces in the applicationUpload files with tracesDashboardsLog traces to specific projectSet up automation rulesOnline EvaluationSet a sampling rate for tracesAdd metadata and tags to tracesImplement distributed tracingAccess the current run (span) within a traced functionLog multimodal tracesLog re

In [46]:
## load data ---> Docs---> divide document into chaunk document---> vecotr-->vectore Embeddign----> vectorestore in DB

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
document  = text_splitter.split_documents(docs)
document

[Document(metadata={'source': 'https://docs.smith.langchain.com/observability/tutorials/observability', 'title': 'Add observability to your LLM application | 🦜️🛠️ LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content='Add observability to your LLM application | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/observability/tutorials/observability', 'title': 'Add observability to your LLM application | 🦜️🛠️ LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content='Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityQuick StartTutorialsAdd observability to your LLM applicationHow-to GuidesAnnotate code for tracingFilter t

In [47]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings()

In [48]:
from langchain_community.vectorstores import FAISS
vecotrStoredb = FAISS.from_documents(document,embedding)

In [49]:
# input ----> Retriever ----> vectorestoredb
vecotrStoredb

In [50]:
## Query from vectorstore db
query = "Observability is important for any software application, but especially so for LLM applications."
result = vecotrStoredb.similarity_search(query)
result[0].page_content

'Add observability to your LLM application | 🦜️🛠️ LangSmith'

In [51]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),
    model_name="llama3-70b-8192"
)

In [52]:
## Retriver Chain , Document Chain 
# "create_stuff_documents_chain":=is context information given of any document 

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the following question based only on the provided context."),
                ("user", "<context>{context}</context>\n\nQuestion: {input}")
    ]
)

document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Answer the following question based only on the provided context.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='<context>{context}</context>\n\nQuestion: {input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002A1801D3A90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002A1801D3490>, model_name='llama3-70b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser

In [53]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"Observability is important for any software application, but especially so for LLM applications.",
    "context":[Document(page_content="Observability is important for any software application, but especially so for LLM applications.")]
})

'True.'

#### Ye code ek document ko chain me pass karta hai. Chain document ke text ko context ke roop me LLM ko bhejta hai, aur phir LLM us context ke base par answer generate karke return karta hai.

In [54]:
### input ----> Retriever ----> vectorsotredb
vecotrStoredb

In [55]:
retriever =vecotrStoredb.as_retriever()

from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever, document_chain)

In [59]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002A1801D3BE0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Answer the following question based only on the provided context.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(in

In [63]:
### GEt the responce from the LLM
response = retrieval_chain.invoke({
    "input":"Observability is important for any software application, but especially so for LLM applications."
})
print(response["answer"])

True. According to the context, observability is important for any software application, but it is especially crucial for LLM applications because they are non-deterministic by nature, meaning they can produce unexpected results, making them trickier to debug.


In [61]:
response

{'input': 'Observability is important for any software application, but especially so for LLM applications.',
 'context': [Document(id='354ca7a2-fcb9-4d8f-aa6b-0b437b7ed196', metadata={'source': 'https://docs.smith.langchain.com/observability/tutorials/observability', 'title': 'Add observability to your LLM application | 🦜️🛠️ LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content='Add observability to your LLM application | 🦜️🛠️ LangSmith'),
  Document(id='e16a78e4-c258-4ed0-b82b-69f9f6c324ac', metadata={'source': 'https://docs.smith.langchain.com/observability/tutorials/observability', 'title': 'Add observability to your LLM application | 🦜️🛠️ LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content="Observability is important for any software application, but especially so for LLM 

In [62]:
response['context']

[Document(id='354ca7a2-fcb9-4d8f-aa6b-0b437b7ed196', metadata={'source': 'https://docs.smith.langchain.com/observability/tutorials/observability', 'title': 'Add observability to your LLM application | 🦜️🛠️ LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content='Add observability to your LLM application | 🦜️🛠️ LangSmith'),
 Document(id='e16a78e4-c258-4ed0-b82b-69f9f6c324ac', metadata={'source': 'https://docs.smith.langchain.com/observability/tutorials/observability', 'title': 'Add observability to your LLM application | 🦜️🛠️ LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content="Observability is important for any software application, but especially so for LLM applications.\nLLMs are non-deterministic by nature, meaning they can produce unexpected results.\nThis makes them trickier